In [45]:
%load_ext autoreload
%autoreload 2
import xarray as xr 
import numpy as np  
import cftime
import copy
import scipy.stats
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as xccrs
import cartopy.feature as cfeature
import xesmf as xe
%matplotlib inline

import momlevel

import matplotlib.ticker as mticker
panellabs=list(map(chr, range(97, 123)))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data I/O using Dask: probably necessary if more than ~30 tgs

In [44]:
cluster.close()
client.close()
client.shutdown()

In [36]:
def get_ClusterClient():
    import dask
    from dask_jobqueue import PBSCluster
    from dask.distributed import Client
    cluster = PBSCluster(
        cores=1,
        memory='20GB',
        processes=1,
        queue='casper',
        resource_spec='select=1:ncpus=1:mem=20GB',
        project='UARH0002',
        walltime='02:00:00',
        interface='ib0',)

    dask.config.set({
        'distributed.dashboard.link':
        'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status',
        'array.slicing.split_large_chunks': True
    })
    client = Client(cluster)
    return cluster, client

cluster, client = get_ClusterClient()
cluster.scale(20) 

/glade/work/clittle/conda-envs/sealevel-newmomlevel/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile
/glade/work/clittle/conda-envs/sealevel-newmomlevel/lib/python3.8/site-packages/distributed/node.py:177: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32925 instead
  warnings.warn(


In [37]:
cluster

## altimetry import to monthly: needs to be done in stages; only if you need beyond 2018

In [2]:
# datadir = '/glade/work/clittle/altimetry/MEASURES/'
# years=np.arange(1993,1996,1)
# months=np.arange(1,13,1)#13,1)
# datasets = []
# Y=1993
# d0 = xr.open_mfdataset(datadir+'ssh_grids_v2205_'+str(201)+'*.nc')
# alt_mo = d0.resample(Time="M").mean() 
# dest_dir = '/glade/work/clittle/altimetry/MEASURES/monthly/'
# alt_mo.to_netcdf(dest_dir+'monthly_2010s.nc')

## process ALT at TG locations

note no correction for global mean terms, and/or many other missing processes

In [39]:
tgs_in=pd.read_csv("real_tgs_in.csv")
tg_ibcorr=pd.read_pickle("tg_ts_93_18.pkl")

datadir = '/glade/work/clittle/altimetry/MEASURES/monthly/'
alt=xr.open_mfdataset(datadir+'monthly_*.nc')
alt=alt*100
alt

<xarray.Dataset>
Dimensions:     (Time: 312, Longitude: 2160, Latitude: 960, nv: 2)
Coordinates:
  * Time        (Time) datetime64[ns] 1993-01-31 1993-02-28 ... 2018-12-31
  * Longitude   (Longitude) float32 0.08333 0.25 0.4167 ... 359.6 359.8 359.9
  * Latitude    (Latitude) float32 -79.92 -79.75 -79.58 ... 79.58 79.75 79.92
Dimensions without coordinates: nv
Data variables:
    Lon_bounds  (Time, Longitude, nv) float32 dask.array<chunksize=(84, 2160, 2), meta=np.ndarray>
    Lat_bounds  (Time, Latitude, nv) float32 dask.array<chunksize=(84, 960, 2), meta=np.ndarray>
    SLA         (Time, Latitude, Longitude) float32 dask.array<chunksize=(84, 960, 2160), meta=np.ndarray>
    SLA_ERR     (Time, Latitude, Longitude) float32 dask.array<chunksize=(84, 960, 2160), meta=np.ndarray>

In [40]:
geolon=alt.Longitude
geolat=alt.Latitude
omask=xr.where(np.isnan(alt.SLA[0,:,:]), 0, 1)
alt_tg_ts=momlevel.extract_tidegauge(alt.SLA, geolon, geolat, mask=omask, csv="./real_tgs_in.csv")

/glade/work/clittle/conda-envs/sealevel-newmomlevel/lib/python3.8/site-packages/momlevel/util.py:514: UserWarning: Constructing coordinates from 1-D vectors. Make sure this is the intended behavior. Do not use `xh`/`yh` when `geolon`/`geolat` are available
  warnings.warn(


In [41]:
alt_in=tgs_in.copy()
alt_in.set_index('name', inplace=True)
alt_in=alt_in.T

for colname in alt_in:
    alt_in[colname].lat=alt_tg_ts[colname].model_coords[0]
    alt_in[colname].lon=alt_tg_ts[colname].model_coords[1]
    # print(colname)
# alt_in.to_pickle("./alt_pseudo_tg_locs.pkl")
alt_in.to_pickle("./alt_real_tg_locs.pkl")

In [42]:
alt_ibcorr=alt_tg_ts.to_dataframe()
alt_ibcorr=alt_ibcorr.T
alt_ibcorr.index=alt_ibcorr.index.astype('str')
alt_ibcorr=alt_ibcorr.T
alt_ibcorr.columns
alt_ibcorr.index=tg_ibcorr.index
alt_ibcorr.to_pickle("./real_alt_ts.pkl")